In [1]:
import sys
from pathlib import Path
path_root = '../'
sys.path.append(str(path_root))

In [2]:
import numpy as np
import pandas as pd
from one.api import ONE
from brainbox.io.one import SessionLoader, SpikeSortingLoader
from iblatlas.atlas import AllenAtlas
from utils.ibl_data_loading import (
    load_spiking_data, load_trials_and_mask, merge_probes,
    load_discrete_behaviors, load_continuous_behaviors
)
from utils.ibl_data_processing import prepare_data

/home/yizi/anaconda3/envs/ibl_foundation_model/lib/python3.11/site-packages/neurodsp/__init__.py:5: FutureWarning: neurodsp has been renamed to ibldsp and the old name will be deprecated on 01-Sep-2024.
  warn(


##### Data Loading Example (Single Session)

In [3]:
# Instantiate ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', 
          silent=True, 
          cache_dir='/mnt/3TB/yizi/Downloads/ONE/openalyx.internationalbrainlab.org'
)
one = ONE(password='international')
ba = AllenAtlas()

/home/yizi/anaconda3/envs/ibl_foundation_model/lib/python3.11/site-packages/one/params.py:184: UserWarning: Warning: the directory provided is already a cache for another URL.
  warnings.warn('Warning: the directory provided is already a cache for another URL.')


In [4]:
# List all publicly available sessions
# sessions = one.search()

# List all sessions from the brain-wide-map project
# Note: Not all sessions will have behaviors recorded so we can use sessions 
#       w/ behaviors for supervised tasks and those w/o for SSL
# sessions = one.search(project='brainwide')

# List brainwide map sessions that pass the most important quality controls
# Note: Let's first work with these good quality sessions, and 
#       transition to using more sessions later on
freeze_file = '../data/2023_12_bwm_release.csv'
bwm_df = pd.read_csv(freeze_file, index_col=0)

In [5]:
# Load an example session
idx = 10
pid = bwm_df.pid[idx]
eid, probe = one.pid2eid(pid)

In [6]:
# Load spike sorting data
spikes, clusters = load_spiking_data(one, pid)

In [7]:
# We may not want to train the model with data within trials, but we may need it for eval purposes.
# Load trials data and mask. Trials are excluded in the mask if reaction time is too long or too short,
# or if critical trial events are missing.
trials, mask = load_trials_and_mask(one, eid, min_rt=0.08, max_rt=2., nan_exclude='default')

In [8]:
# Load discrete behaviors for trial decoding
discrete_behaviors = load_discrete_behaviors(trials)
print(discrete_behaviors.keys())

dict_keys(['choice', 'block', 'reward', 'contrast'])


In [9]:
# Load continuous behaviors for any-time decoding
continuous_behaviors = load_continuous_behaviors(one, eid)
print(continuous_behaviors.keys())

dict_keys(['wheel_pos', 'wheel_vel', 'left_whisker', 'right_whisker', 'body_whisker', 'left_pupil', 'right_pupil', 'left_paw_l', 'left_paw_r', 'right_paw_l', 'right_paw_r', 'left_nose', 'right_nose'])


##### Prepare Data for Any-Time Decoding (Single Session)

In [10]:
params = {
    'merged_probes': True
}

In [11]:
neural_dict, behave_dict, metadata = prepare_data(one, idx, bwm_df, params)

Running merged probes for session eid: 5ec72172-3901-4771-8777-6e9490ca51fc


##### Prepare Data for Trial Decoding (In Progress)

##### Download and Cache Data  (Multiple Sessions)